<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-2">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-3">Main</a></span><ul class="toc-item"><li><span><a href="#CBTTC" data-toc-modified-id="CBTTC-3.1">CBTTC</a></span></li><li><span><a href="#TARGET" data-toc-modified-id="TARGET-3.2">TARGET</a></span></li><li><span><a href="#TCGA" data-toc-modified-id="TCGA-3.3">TCGA</a></span></li><li><span><a href="#DKFZ" data-toc-modified-id="DKFZ-3.4">DKFZ</a></span></li><li><span><a href="#Bind-all!" data-toc-modified-id="Bind-all!-3.5">Bind all!</a></span></li><li><span><a href="#Clinical-data-cleanup" data-toc-modified-id="Clinical-data-cleanup-3.6">Clinical data cleanup</a></span></li></ul></li></ul></div>

# Dependencies

In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Paths

In [2]:
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Manifests/"
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Plots/"

# Main

## CBTTC

This file is from CBTTC Dataset - Arash Nabbi 12.04.18.xlsx from J.Mason spreadsheet "Clinical Dataset wout aliquots"

In [3]:
CBTTC_clinical <- read.csv(paste0(manifestpath,"CBTTC_clinical.csv"),header = TRUE, 
                           stringsAsFactors = FALSE, na.strings = c("", NA))

In [4]:
CBTTC_clinical <- CBTTC_clinical[!is.na(CBTTC_clinical$CBTTC.Subject.ID),]

In [5]:
CBTTC_clinical$sample_id <- CBTTC_clinical$CBTTC.Event.ID

In [6]:
CBTTC_clinical$Age.at.Collection[CBTTC_clinical$Age.at.Collection == "Not Reported"] <- NA
CBTTC_clinical$Age.At.Last.Known.Status..if.deceased..this.is.days.to.death.[CBTTC_clinical$Age.At.Last.Known.Status..if.deceased..this.is.days.to.death. == "Not reported"] <- NA

In [7]:
CBTTC_clinical$Age.at.Collection <- as.numeric(CBTTC_clinical$Age.at.Collection)
CBTTC_clinical$Age.At.Last.Known.Status..if.deceased..this.is.days.to.death. <- as.numeric(CBTTC_clinical$Age.At.Last.Known.Status..if.deceased..this.is.days.to.death.)

make days to death variable

In [8]:
CBTTC_clinical$days_to_death <- NA

In [9]:
cbttc_caseids <- unique(CBTTC_clinical$CBTTC.Subject.ID)

for each caseid, if only one entry days to death = ageatlastknown - ageatcollection
if more than one entry, days to death = maximum age at last known - minimum age at collection


this is to capture days to death for progressive samples

In [10]:
for(id in cbttc_caseids){
    tmp <- CBTTC_clinical[ CBTTC_clinical$CBTTC.Subject.ID == id,]
    if(nrow(tmp) == 1){
        CBTTC_clinical$days_to_death[CBTTC_clinical$CBTTC.Subject.ID == id] <- tmp$`Age.At.Last.Known.Status..if.deceased..this.is.days.to.death.` - tmp$Age.at.Collection    
    }
    else{
    age.at.firstcollection <- min(tmp$Age.at.Collection)
    age.at.last.known <- max(tmp$`Age.At.Last.Known.Status..if.deceased..this.is.days.to.death.`)
    CBTTC_clinical$days_to_death[CBTTC_clinical$CBTTC.Subject.ID == id] <-  age.at.last.known - age.at.firstcollection   
        
    }
}

make a recurrence variable and days to progress

In [11]:
CBTTC_clinical$recurrence <- NA
CBTTC_clinical$days_to_progress <- NA

for each caseid, if only one entry: recurrence == lastknown status, days to progress is same as days to death
    
for more than one entry, recurred, order based on age at collection, days to progress (days to first event) is difference between the first entry and the second entry

In [12]:
for(id in cbttc_caseids){
    tmp <- CBTTC_clinical[ CBTTC_clinical$CBTTC.Subject.ID == id,]
    if(nrow(tmp) == 1){
        CBTTC_clinical$recurrence[CBTTC_clinical$CBTTC.Subject.ID == id] <- tmp$Last.Known.Status
        CBTTC_clinical$days_to_progress[CBTTC_clinical$CBTTC.Subject.ID == id] <- CBTTC_clinical$days_to_death[CBTTC_clinical$CBTTC.Subject.ID == id]
    
    }
    else{
        #print(tmp$CBTTC.Subject.ID)
        CBTTC_clinical$recurrence[CBTTC_clinical$CBTTC.Subject.ID == id] <- 1 #if more than one entry, recurred
        #order based on age at collection
        tmp <- tmp[order(tmp$Age.at.Collection),]
        progressed_time <- tmp[2,"Age.at.Collection"] - tmp[1,"Age.at.Collection"]
        CBTTC_clinical$days_to_progress[CBTTC_clinical$CBTTC.Subject.ID == id] <- progressed_time
        
    }
}

In [13]:
CBTTC_clinical[CBTTC_clinical$Last.Known.Status == "Deceased-due to disease" & is.na(CBTTC_clinical$days_to_progress),]

,CBTTC.Subject.ID,CBTTC.Event.ID,Kids.Subject.ID,Group,Cohort,Sample.Clinical.Event.Origin,Gender,Race,Ethnicity,Diagnosis.a,⋯,Primary.Site.e,Primary.Site.f,Primary.Site.g,Age.at.Collection,Last.Known.Status,Age.At.Last.Known.Status..if.deceased..this.is.days.to.death.,sample_id,days_to_death,recurrence,days_to_progress
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
75,C129273,7316-839,PT_5JA1N2H4,CBTTC,MB,Progressive,Male,Other,Reported Unknown,Medulloblastoma,⋯,Not Applicable,Not Applicable,Not Applicable,NA,Deceased-due to disease,NA,7316-839,NA,Deceased-due to disease,NA
369,C372813,7316-1889,PT_KBDP8ZCR,CBTTC,HGG,Progressive,Male,Other,Reported Unknown,High-grade glioma/astrocytoma (WHO grade III/IV),⋯,Not Applicable,Not Applicable,Not Applicable,6692,Deceased-due to disease,NA,7316-1889,NA,Deceased-due to disease,NA


In [14]:
table(CBTTC_clinical$Last.Known.Status)


                         Alive    Deceased-causes unavailable 
                           826                              2 
       Deceased-due to disease   Deceased-due to other causes 
                           199                              5 
Deceased-due to unknown causes               Reported Unknown 
                             5                             25 

In [15]:
table(CBTTC_clinical$recurrence)


                             1                          Alive 
                           197                            704 
   Deceased-causes unavailable        Deceased-due to disease 
                             2                            124 
  Deceased-due to other causes Deceased-due to unknown causes 
                             5                              5 
              Reported Unknown 
                            25 

If Deceased-due to disease, recurrence = yes. recurrence is disease-specific so censor death due to other causes

In [16]:
CBTTC_clinical$recurrence[ CBTTC_clinical$recurrence == "Deceased-due to disease"] <- 1

CBTTC_clinical$recurrence[ CBTTC_clinical$recurrence == "Alive"] <- 0
CBTTC_clinical$recurrence[ CBTTC_clinical$recurrence == "Deceased-due to unknown causes"] <- 0
CBTTC_clinical$recurrence[ CBTTC_clinical$recurrence == "Deceased-due to other causes"] <- 0
CBTTC_clinical$recurrence[ CBTTC_clinical$recurrence == "Deceased-causes unavailable"] <- 0
CBTTC_clinical$recurrence[ CBTTC_clinical$recurrence == "Reported Unknown"] <- NA

This is overall survival so recode death due to other causes to 1

In [17]:
CBTTC_clinical$Last.Known.Status[ CBTTC_clinical$Last.Known.Status == "Deceased-due to disease"] <- 1

CBTTC_clinical$Last.Known.Status[ CBTTC_clinical$Last.Known.Status == "Alive"] <- 0
CBTTC_clinical$Last.Known.Status[ CBTTC_clinical$Last.Known.Status == "Deceased-due to unknown causes"] <- 1
CBTTC_clinical$Last.Known.Status[ CBTTC_clinical$Last.Known.Status == "Deceased-due to other causes"] <- 1
CBTTC_clinical$Last.Known.Status[ CBTTC_clinical$Last.Known.Status == "Deceased-causes unavailable"] <- 1

#based on osvaldo recommendation, there is no last follow up data so recode to NA
CBTTC_clinical$Last.Known.Status[ CBTTC_clinical$Last.Known.Status == "Reported Unknown"] <- NA

Metadata manifest for CBTTC RSEM files exported from CAVATICA. lines and dups removed

In [18]:
CBTTC_RNA_manifest_genes <- read.csv(paste0(manifestpath, "CBTTC_RSEM-manifest_linerm_dedup.csv"),
                               header = TRUE, stringsAsFactors = FALSE)

In [19]:
dim(CBTTC_RNA_manifest_genes)

[1] 969  20

In [20]:
table(CBTTC_RNA_manifest_genes$Tumor.Descriptor)


                  Initial CNS Tumor       Progressive        Recurrence 
                1               700               147                68 
Second Malignancy       Unavailable 
               16                37 

In [21]:
table(CBTTC_clinical$Sample.Clinical.Event.Origin)


Initial CNS Tumor       Progressive        Recurrence Second Malignancy 
              807               164                73                18 

In [22]:
dim(CBTTC_clinical)

[1] 1062   27

In [23]:
CBTTC_metadata <- merge(CBTTC_clinical, CBTTC_RNA_manifest_genes[c("sample_id", "Kids.First.Biospecimen.ID",
                                                                   "aliquot_id","name")],by = "sample_id")

In [24]:
colnames(CBTTC_RNA_manifest_genes)

[1] "id"                        "name"                     
 [3] "project"                   "visible"                  
 [5] "Kids.First.Participant.ID" "ethnicity"                
 [7] "gender"                    "race"                     
 [9] "disease_type"              "sample_id"                
[11] "Tumor.Descriptor"          "sample_type"              
[13] "platform"                  "Kids.First.Biospecimen.ID"
[15] "primary_site"              "age_at_diagnosis"         
[17] "aliquot_id"                "reference_genome"         
[19] "case_id"                   "experimental_strategy"

In [25]:
# This column originates from clinical excel sheet. Appears to be updated more than cavatica manifest file. Use this instead of Tumor.Descriptor
table(CBTTC_metadata$Sample.Clinical.Event.Origin)


Initial CNS Tumor       Progressive        Recurrence Second Malignancy 
              730               153                70                16 

Some minor cleanup for cohorts

In [26]:
CBTTC_metadata$Cohort[CBTTC_metadata$Cohort == "LGG"] <- "pedLGG"
CBTTC_metadata$Cohort[CBTTC_metadata$Cohort == "HGG"] <- "pedHGG"
CBTTC_metadata$Cohort[CBTTC_metadata$Cohort == "SchN"] <- "SCHN"

Add diagnosis.for.other to diagnosis.a

In [27]:
CBTTC_metadata$Diagnosis.for.Other[CBTTC_metadata$Diagnosis.for.Other == "Not applicable"] <- NA

In [28]:
CBTTC_metadata$Diagnosis.for.Other <- as.character(CBTTC_metadata$Diagnosis.for.Other)
CBTTC_metadata$Diagnosis.a <- as.character(CBTTC_metadata$Diagnosis.a)

In [29]:
CBTTC_metadata$Diagnosis.a[!is.na(CBTTC_metadata$Diagnosis.for.Other)] <- CBTTC_metadata$Diagnosis.for.Other[!is.na(CBTTC_metadata$Diagnosis.for.Other)]

In [30]:
CBTTC_metadata$Diagnosis.for.Other <- NULL

Convert age from days to years and round

In [31]:
CBTTC_metadata$age_at_diagnosis <- round(CBTTC_metadata$Age.at.Collection / 365)

Change colnames to bind with other files

In [32]:
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "CBTTC.Subject.ID"] <- "case_id"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "Group"] <- "group"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "Cohort"] <- "cohort" 
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "Sample.Clinical.Event.Origin"] <- "sample_type"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "Gender"] <- "gender"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "Race"] <- "race"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "Ethnicity"] <- "ethnicity"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "Diagnosis.a"] <- "disease_type"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "Primary.Site.a"] <- "primary_site"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "age_at_diagnosis"] <- "age_at_diagnosis"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "Last.Known.Status"] <- "vital_status"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "days_to_death"] <- "days_to_death"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "CBTTC.Aliquot.ID"] <- "aliquot_id"
colnames(CBTTC_metadata)[colnames(CBTTC_metadata) == "name"] <- "name"

Further clean up for cohorts and CRC rev in cohort_cleanup nb

## TARGET

Metadata manifest for TARGET RNAseq bam files exported from CAVATICA

In [33]:
TARGET_RNA_manifest <- read.csv(paste0(manifestpath, "TARGET-manifest.csv"),header = TRUE, stringsAsFactors = FALSE)

In [34]:
TARGET_RNA_manifest$group <- "TARGET"
TARGET_RNA_manifest$cohort <- NA
TARGET_RNA_manifest$Sample <- NA
TARGET_RNA_manifest$recurrence <- NA
TARGET_RNA_manifest$days_to_progress <- NA

Extract sample type from sample_id and rename

In [35]:
TARGET_RNA_manifest$Sample <- substr(TARGET_RNA_manifest$sample_id, nchar(TARGET_RNA_manifest$sample_id)-2, nchar(TARGET_RNA_manifest$sample_id)-1)

In [36]:
TARGET_RNA_manifest$Sample[TARGET_RNA_manifest$Sample == "01"] <- "Primary Solid Tumor"
TARGET_RNA_manifest$Sample[TARGET_RNA_manifest$Sample == "02"] <- "Recurrent Solid Tumor"
TARGET_RNA_manifest$Sample[TARGET_RNA_manifest$Sample == "04"] <- "Recurrent BM/Blood"
TARGET_RNA_manifest$Sample[TARGET_RNA_manifest$Sample == "11"] <- "Solid Tissue Normal"
TARGET_RNA_manifest$Sample[TARGET_RNA_manifest$Sample == "06"] <- "Metastatic"

In [37]:
TARGET_RNA_manifest$cohort[TARGET_RNA_manifest$disease_type == "Neuroblastoma"] <- "NBL"
TARGET_RNA_manifest$cohort[TARGET_RNA_manifest$disease_type == "Rhabdoid Tumor"] <- "RT"
TARGET_RNA_manifest$cohort[TARGET_RNA_manifest$disease_type == "High-Risk Wilms Tumor"] <- "WILMS"

In [38]:
table(TARGET_RNA_manifest$cohort)


  NBL    RT WILMS 
  157     5   132 

Remove RT and WILMS

In [39]:
TARGET_RNA_manifest <- TARGET_RNA_manifest[TARGET_RNA_manifest$cohort == "NBL",]

In [40]:
dim(TARGET_RNA_manifest)

[1] 157  28

In [41]:
table(TARGET_RNA_manifest$Sample)


  Primary Solid Tumor    Recurrent BM/Blood Recurrent Solid Tumor 
                  149                     1                     7 

In [42]:
colnames(TARGET_RNA_manifest)[colnames(TARGET_RNA_manifest) == "Sample"] <- "sample_type"

Add OS and PFS from NBL clinical data that I downloaded from TARGET portal

In [43]:
NBL_manifest <- read.table(paste0(manifestpath, "TARGET_NBL_Discovery_ClinicalData_20170525.txt"),
                                 sep = "\t", na.strings = "", header = T, stringsAsFactors = F)

Subset to only samples in KF dataset

In [44]:
NBL_TARGETinKF <- NBL_manifest[NBL_manifest$TARGET.USI %in% TARGET_RNA_manifest$case_id,]

Match case ids and fill OS and PFS and disease_type

In [45]:
TARGET_RNA_manifest$days_to_death[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)] <-
NBL_TARGETinKF$Overall.Survival.Time.in.Days[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)]

In [46]:
TARGET_RNA_manifest$days_to_progress[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)] <-
NBL_TARGETinKF$Event.Free.Survival.Time.in.Days[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)]

In [47]:
TARGET_RNA_manifest$disease_type[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)] <-
NBL_TARGETinKF$Diagnostic.Category[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)]

if first event is Event or relapsed or progress or death it means it recurred, if censored no recurrence

In [48]:
TARGET_RNA_manifest$recurrence[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)] <-
NBL_TARGETinKF$First.Event[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)]

In [49]:
head(NBL_TARGETinKF[,20:30])

,Diagnostic.Category,ICDO,ICDO.Description,COG.Risk.Group,Percent.Tumor,Percent.Necrosis,Percent.Tumor.v.s.Stroma,Sites.of.Disease.Involvement,Site.Relapse,Relapse.Percent.Tumor,Relapse.Percent.Necrosis
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
79,Unknown,Unknown,ABDOMEN/PERITONEUM OR RETROPERITONEUM,High Risk,NA,NA,NA,NA,NA,NA,NA
91,Unknown,Unknown,ADRENAL GLAND,High Risk,90,10,95/5,NA,NA,NA,NA
100,Unknown,Unknown,ADRENAL GLAND,High Risk,80-90,5,NA,NA,NA,NA,NA
102,Neuroblastoma,Unknown,ADRENAL GLAND,High Risk,90,10,90/10,NA,NA,NA,NA
109,Neuroblastoma,C74.9,"Adrenal gland, NOS Suprarenal gland Adrenal, NOS",High Risk,90,10,70/30,NA,NA,NA,NA
120,"Ganglioneuroblastoma, nodular",Unknown,ADRENAL GLAND,High Risk,NA,NA,NA,NA,NA,NA,NA


In [50]:
table(TARGET_RNA_manifest$recurrence)


                 Censored                     Death                     Event 
                       54                        14                        15 
              Progression                   Relapse Second Malignant Neoplasm 
                       19                        42                         1 

In [51]:
TARGET_RNA_manifest$recurrence[TARGET_RNA_manifest$recurrence %in% c("Event", "Death", "Progression", "Relapse")] <- 1
TARGET_RNA_manifest$recurrence[TARGET_RNA_manifest$recurrence == "Censored"] <- 0
TARGET_RNA_manifest$recurrence[TARGET_RNA_manifest$recurrence == "Second Malignant Neoplasm"] <- NA

some information regarding the tumor site is in ICDO description, add and cleanup

In [52]:
TARGET_RNA_manifest$primary_site[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)] <-
NBL_TARGETinKF$ICDO.Description[match(NBL_TARGETinKF$TARGET.USI, TARGET_RNA_manifest$case_id)]

In [53]:
TARGET_RNA_manifest$primary_site[grepl("Adrenal gland",TARGET_RNA_manifest$primary_site, ignore.case = T)] <- "Adrenal gland"
TARGET_RNA_manifest$primary_site[grepl("Retroperitoneum",TARGET_RNA_manifest$primary_site, ignore.case = T)] <- "Retroperitoneum"
TARGET_RNA_manifest$primary_site[grepl("Abdomen",TARGET_RNA_manifest$primary_site, ignore.case = T)]  <- "Abdomen"
TARGET_RNA_manifest$primary_site[grepl("Abdominal",TARGET_RNA_manifest$primary_site, ignore.case = T)]  <- "Abdomen"
TARGET_RNA_manifest$primary_site[grepl("Chest",TARGET_RNA_manifest$primary_site, ignore.case = T)] <- "Chest"
TARGET_RNA_manifest$primary_site[grepl("mediastinum",TARGET_RNA_manifest$primary_site, ignore.case = T)] <- "Mediastinum"
TARGET_RNA_manifest$primary_site[grepl("Kidney",TARGET_RNA_manifest$primary_site, ignore.case = T)] <- "Kidney"
TARGET_RNA_manifest$primary_site[grepl("Trunk",TARGET_RNA_manifest$primary_site, ignore.case = T)] <- "Trunk"
TARGET_RNA_manifest$primary_site[grepl("Pelvis",TARGET_RNA_manifest$primary_site, ignore.case = T)] <- "Pelvis"
TARGET_RNA_manifest$primary_site[grepl("liver",TARGET_RNA_manifest$primary_site, ignore.case = T)] <- "Liver"

## TCGA

Metadata manifest for TCGA RNAseq bam files exported from CAVATICA

In [54]:
TCGA_manifest <- read.csv(paste0(manifestpath,"TCGA-manifest.csv"),
                             header = TRUE, stringsAsFactors = FALSE)

Remove .bai files

In [55]:
TCGA_manifest <- TCGA_manifest[grepl(".bam",TCGA_manifest$name),]

In [56]:
dim(TCGA_manifest)

[1] 3223   23

In [57]:
TCGA_manifest$group <- "TCGA"
TCGA_manifest$cohort <- NA
TCGA_manifest$Sample <- NA

TCGA_manifest$days_to_progress <- NA
TCGA_manifest$recurrence <- NA

Extract sample type from sample_id and rename

In [58]:
TCGA_manifest$Sample <- substr(TCGA_manifest$sample_id, nchar(TCGA_manifest$sample_id)-2, nchar(TCGA_manifest$sample_id)-1)

In [59]:
table(TCGA_manifest$Sample)


  01   02   06   07   11 
2656   39  369    1  158 

In [60]:
TCGA_manifest$Sample[TCGA_manifest$Sample == "01"] <- "Primary Solid Tumor"
TCGA_manifest$Sample[TCGA_manifest$Sample == "02"] <- "Recurrent Solid Tumor"
TCGA_manifest$Sample[TCGA_manifest$Sample == "04"] <- "Recurrent BM/Blood"
TCGA_manifest$Sample[TCGA_manifest$Sample == "11"] <- "Solid Tissue Normal"
TCGA_manifest$Sample[TCGA_manifest$Sample == "06"] <- "Metastatic"
TCGA_manifest$Sample[TCGA_manifest$Sample == "07"] <- "Additional Metastatic"

In [61]:
TCGA_manifest$cohort[TCGA_manifest$disease_type == "Brain Lower Grade Glioma"] <- "LGG"
TCGA_manifest$cohort[TCGA_manifest$disease_type == "Colon Adenocarcinoma"] <- "COAD"
TCGA_manifest$cohort[TCGA_manifest$disease_type == "Lung Adenocarcinoma"] <- "LUAD"
TCGA_manifest$cohort[TCGA_manifest$disease_type == "Glioblastoma Multiforme"] <- "GBM"
TCGA_manifest$cohort[TCGA_manifest$disease_type == "Ovarian Serous Cystadenocarcinoma"] <- "OV"
TCGA_manifest$cohort[TCGA_manifest$disease_type == "Skin Cutaneous Melanoma"] <- "SKCM"
TCGA_manifest$cohort[TCGA_manifest$disease_type == "Prostate Adenocarcinoma"] <- "PRAD"

In [62]:
dim(TCGA_manifest)

[1] 3223   28

Remove duplicates

In [63]:
TCGA_manifest <- TCGA_manifest[!duplicated(TCGA_manifest$sample_id),]

In [64]:
colnames(TCGA_manifest)[colnames(TCGA_manifest) == "Sample"] <- "sample_type"

## DKFZ


this file is from Natalie

In [65]:
DKFZ_manifest <- read.csv(paste0(manifestpath,"DKFZ_clinical.csv"), header = T, stringsAsFactors = F)

In [66]:
dim(DKFZ_manifest)

[1] 200   7

Survival data from Pengbo

In [67]:
head(DKFZ_manifest)

,SampleID,Cancer.Type,Subgroup,Type,Relapse.of.Other.Sample,Age,Gender
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,H049-UV6K,ATRT,MYC,Primary,x,NA,Male
2,H049-0CWK,ATRT,MYC,Primary,x,4.0,Male
3,H049-FG38,ATRT,SHH,Primary,x,1.0,Female
4,H049-GP41,ATRT,SHH,Primary,x,1.0,Male
5,H049-GWNW,ATRT,SHH,Primary,x,1.7,Female
6,H049-JVCT,ATRT,SHH,Primary,x,3.0,Female


In [68]:
DKFZ_survival <- read.csv(paste0(manifestpath,"DKFZ_OS.csv"), header = T, stringsAsFactors = F)

In [69]:
table(DKFZ_survival$Recurrence)


                                                    0 
                       280                        167 
                         1 R; primary tumor - 4602/10 
                        63                          1 
R; primary tumor - 6211/10 
                         1 

In [70]:
head(DKFZ_survival)

,SampleID_ICGC,Location,Death,OS,Recurrence,PFS
,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,ICGC_PA127,Optic nerve,0,240,0,240
2,ICGC_PA26,Cerebellum,0,206,1,64
3,ICGC_PA27,Optic nerve,0,219,0,219
4,ICGC_PA28,Cerebellum,0,1,0,1
5,ICGC_PA29,Hypothalamus,0,223,0,223
6,ICGC_PA128,Cerebellum,0,224,0,224


In [71]:
colnames(DKFZ_survival)[colnames(DKFZ_survival) == "SampleID_ICGC"] <- "SampleID"

In [72]:
DKFZ_manifest <- merge(DKFZ_manifest, DKFZ_survival, by = "SampleID", all.x = T)

In [73]:
dim(DKFZ_manifest)

[1] 200  12

In [74]:
DKFZ_manifest$Death[DKFZ_manifest$Death == ""] <- NA

In [75]:
table(DKFZ_manifest$Recurrence)


    0  1 
89 58 13 

Add columns to be consistent with other datasets

In [76]:
DKFZ_manifest$case_id <- DKFZ_manifest$SampleID
DKFZ_manifest$aliquot_id <- DKFZ_manifest$SampleID
DKFZ_manifest$race <- NA
DKFZ_manifest$ethnicity <- NA
DKFZ_manifest$group <- "DKFZ"
DKFZ_manifest$vital_status <- DKFZ_manifest$Death
# Survival data is in months
DKFZ_manifest$days_to_death <- DKFZ_manifest$OS * 30
DKFZ_manifest$days_to_progress <- DKFZ_manifest$PFS * 30

DKFZ_manifest$disease_type <- paste(DKFZ_manifest$Cancer.Type, DKFZ_manifest$Subgroup, sep = "_")
DKFZ_manifest$primary_site <- DKFZ_manifest$Location

In [77]:
DKFZ_manifest$recurrence <- DKFZ_manifest$Recurrence

In [78]:
DKFZ_manifest$recurrence[DKFZ_manifest$recurrence == ""] <- NA

In [79]:
table(DKFZ_manifest$recurrence)


 0  1 
58 13 

In [80]:
colnames(DKFZ_manifest)[which(names(DKFZ_manifest) == "SampleID")] <- "sample_id"
colnames(DKFZ_manifest)[which(names(DKFZ_manifest) == "Cancer.Type")] <- "cohort"
colnames(DKFZ_manifest)[which(names(DKFZ_manifest) == "Type")] <- "sample_type"
colnames(DKFZ_manifest)[which(names(DKFZ_manifest) == "Age")] <- "age_at_diagnosis"
colnames(DKFZ_manifest)[which(names(DKFZ_manifest) == "Gender")] <- "gender"

In [81]:
DKFZ_manifest$Subgroup <- NULL
DKFZ_manifest$Relapse.of.Other.Sample <- NULL

In [82]:
DKFZ_manifest$cohort[grepl("HGG", DKFZ_manifest$cohort)] <- "pedHGG"
DKFZ_manifest$cohort[grepl("MB", DKFZ_manifest$cohort)] <- "MB"
DKFZ_manifest$cohort[grepl("ETMR", DKFZ_manifest$cohort)] <- "ET"

In [83]:
DKFZ_manifest$disease_type[ DKFZ_manifest$cohort == "pedHGG"]

[1] "HGGother_WT"   "HGGother_IDH"  "HGG_K27M_K27M" "HGG_K27M_K27M"
 [5] "HGGother_IDH"  "HGG_K27M_K27M" "HGG_K27M_K27M" "HGGother_WT"  
 [9] "HGGother_WT"   "HGGother_WT"   "HGGother_WT"   "HGGother_WT"  
[13] "HGGother_G34R" "HGGother_WT"   "HGGother_WT"   "HGG_K27M_K27M"
[17] "HGGother_WT"   "HGGother_WT"   "HGGother_WT"   "HGGother_WT"  
[21] "HGGother_IDH"  "HGGother_G34R" "HGG_K27M_K27M" "HGG_K27M_K27M"
[25] "HGGother_IDH"

Function to add group and cohort variables for DKFZ estimate output. Merging with file with the DKFZ_Manifest just ensures only analyzed samples are included in master file

In [84]:
groupcohortfx <- function(inpath,estimate_outputname_DKFZ, group, cohort){
    estimate_output <- read.table(paste(inpath, estimate_outputname_DKFZ, sep = ""), 
                              sep = "\t", header = TRUE)
    estimate_output$SampleID <- as.character(estimate_output$SampleID)
    estimate_output$group <- group
    estimate_output$cohort <- cohort
    return(estimate_output)
}

In [85]:
ATRT_estimate <- groupcohortfx("/Users/anabbi/OneDrive - UHN/Documents/ImmPedCan/Finaldata/ESTIMATE/",
                              "ICGC_ATRTs.normalizedTPMs_perGencodeV26Gene.sleuth.estimateOutput.txt",
                              "DKFZ", "ATRT")
ET_estimate <- groupcohortfx("/Users/anabbi/OneDrive - UHN/Documents/ImmPedCan/Finaldata/ESTIMATE/",
                              "ICGC_ETs.normalizedTPMs_perGencodeV26Gene.sleuth.estimateOutput.txt",
                              "DKFZ", "ET")
pedGBM_estimate <- groupcohortfx("/Users/anabbi/OneDrive - UHN/Documents/ImmPedCan/Finaldata/ESTIMATE/",
                              "ICGC_GBMs.normalizedTPMs_perGencodeV26Gene.sleuth.estimateOutput.txt",
                              "DKFZ", "pedGBM")
PA_estimate <- groupcohortfx("/Users/anabbi/OneDrive - UHN/Documents/ImmPedCan/Finaldata/ESTIMATE/",
                              "ICGC_PAs.normalizedTPMs_perGencodeV26Gene.sleuth.estimateOutput.txt",
                              "DKFZ", "PA")
MB_estimate <- groupcohortfx("/Users/anabbi/OneDrive - UHN/Documents/ImmPedCan/Finaldata/ESTIMATE/",
                              "ICGC_MBs.normalizedTPMs_perGencodeV26Gene.sleuth.estimateOutput.txt",
                              "DKFZ", "MB")

In [86]:
DKFZ_estimate <- rbind(PA_estimate,ET_estimate,
                      pedGBM_estimate,ATRT_estimate,
                      MB_estimate)

In [87]:
colnames(DKFZ_estimate)[colnames(DKFZ_estimate) == "SampleID"] <- "name"

In [88]:
DKFZ_estimate$name <- gsub("[.]", "-", DKFZ_estimate$name)

In [89]:
DKFZ_estimate$sample_id <- DKFZ_estimate$name

In [90]:
DKFZ_manifest_files <- merge(DKFZ_manifest,DKFZ_estimate[,c("name","sample_id")], by = "sample_id")

In [91]:
DKFZ_manifest$sample_id[!DKFZ_manifest$sample_id %in% DKFZ_manifest_files$sample_id]

[1] "ICGC_MB168"

The case above was not included in estimate analysis

## Bind all!

In [92]:
mycolumns <- 
c("name", "case_id", "sample_id", "aliquot_id", "gender",
  "ethnicity", "race","age_at_diagnosis", "vital_status",
  "days_to_death","days_to_progress", "recurrence",
  "disease_type", "primary_site", "group",
  "cohort", "sample_type")

In [93]:
IPD_metadata <- rbind(TARGET_RNA_manifest[,mycolumns],
                      TCGA_manifest[,mycolumns],
                      CBTTC_metadata[,mycolumns],
                      DKFZ_manifest_files[,mycolumns])

In [94]:
IPD_metadata[is.na(IPD_metadata$recurrence) &IPD_metadata$group != "TCGA",]

,name,case_id,sample_id,aliquot_id,gender,ethnicity,race,age_at_diagnosis,vital_status,days_to_death,days_to_progress,recurrence,disease_type,primary_site,group,cohort,sample_type
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
79,4638324b-b8e5-4545-8b7b-e8d7d7fdd138.bam,TARGET-30-PATYIL,TARGET-30-PATYIL-01A,TARGET-30-PATYIL-01A-01R,female,not hispanic or latino,white,0,dead,NA,NA,NA,Neuroblastoma,Nervous System,TARGET,NBL,Primary Tumor
115,692a3229-af33-4136-bc6e-a4b22f016cde.bam,TARGET-30-PALBFW,TARGET-30-PALBFW-01A,TARGET-30-PALBFW-01A-01R,male,not hispanic or latino,white,2,dead,3264,193,NA,Neuroblastoma,Adrenal gland,TARGET,NBL,Primary Tumor
136,7cccad17-8596-4032-9433-e293f2044d79.bam,TARGET-30-PARBAJ,TARGET-30-PARBAJ-01A,TARGET-30-PARBAJ-01A-01R,male,not hispanic or latino,white,0,alive,NA,NA,NA,Neuroblastoma,Nervous System,TARGET,NBL,Primary Tumor
142,81988869-de07-4b87-ba00-247b93f58732.bam,TARGET-30-PASNPG,TARGET-30-PASNPG-02A,TARGET-30-PASNPG-02A-01R,female,not hispanic or latino,black or african american,0,alive,NA,NA,NA,Neuroblastoma,Nervous System,TARGET,NBL,Recurrent Tumor
232,c7ac5a3e-b090-4882-9dca-94f55e3f7fa4.bam,TARGET-30-PASGAP,TARGET-30-PASGAP-02A,TARGET-30-PASGAP-02A-01R,male,not hispanic or latino,black or african american,3,dead,NA,NA,NA,Neuroblastoma,Nervous System,TARGET,NBL,Recurrent Tumor
255,df7a35a0-dc6a-4801-a962-be2e171eb773.bam,TARGET-30-PAUDDK,TARGET-30-PAUDDK-01A,TARGET-30-PAUDDK-01A-01R,male,not hispanic or latino,black or african american,0,alive,NA,NA,NA,Neuroblastoma,Nervous System,TARGET,NBL,Primary Tumor
266,e99cf13e-9bdf-402d-9ec4-a8bf771543a5.bam,TARGET-30-PARHAM,TARGET-30-PARHAM-01A,TARGET-30-PARHAM-01A-01R,female,not hispanic or latino,white,0,dead,NA,NA,NA,Neuroblastoma,Nervous System,TARGET,NBL,Primary Tumor
283,f7541bf7-73bc-4163-af01-3fea29027a16.bam,TARGET-30-PAPHPE,TARGET-30-PAPHPE-01A,TARGET-30-PAPHPE-01A-01R,male,not hispanic or latino,black or african american,2,dead,NA,NA,NA,NA,NA,TARGET,NBL,Primary Tumor
285,f99e2b20-96d7-49f3-bdd2-7a1cf2fa6208.bam,TARGET-30-PASNZU,TARGET-30-PASNZU-01A,TARGET-30-PASNZU-01A-01R,male,not hispanic or latino,black or african american,3,dead,NA,NA,NA,NA,NA,TARGET,NBL,Primary Tumor


In [95]:
table(IPD_metadata$group)


 CBTTC   DKFZ TARGET   TCGA 
   969    199    157   3204 

## Clinical data cleanup

In [96]:
IPD_metadata$gender[grepl("Reported", IPD_metadata$gender, ignore.case = T)] <- "Unknown"

IPD_metadata$gender[IPD_metadata$gender == "male"] <- "Male"
IPD_metadata$gender[IPD_metadata$gender == "female"] <- "Female"

IPD_metadata$gender[IPD_metadata$gender == "F"] <- "Female"
IPD_metadata$gender[IPD_metadata$gender == "M"] <- "Male"
IPD_metadata$gender[IPD_metadata$gender == "Y"] <- "Reported Unknown"

In [97]:
IPD_metadata$race[IPD_metadata$race == ""] <- NA
IPD_metadata$race[grepl("African",IPD_metadata$race, ignore.case = T)] <- "African"
IPD_metadata$race[grepl("Asian",IPD_metadata$race, ignore.case = T)] <- "Asian"
IPD_metadata$race[grepl("reported",IPD_metadata$race, ignore.case = T)] <- "Unknown"
IPD_metadata$race[grepl("Alaska",IPD_metadata$race, ignore.case = T)] <- "Other"
IPD_metadata$race[grepl("Other",IPD_metadata$race, ignore.case = T)] <- "Other"
IPD_metadata$race[grepl("Pacific",IPD_metadata$race, ignore.case = T)] <- "Other"
IPD_metadata$race[grepl("white",IPD_metadata$race)] <- "White"

In [98]:
table(IPD_metadata$race)


African   Asian   Other Unknown   White 
    301      82     305     655    2978 

clean up vital_status

In [99]:
table(IPD_metadata$vital_status)


                        0            1        alive         dead not reported 
           9          884          201         2206         1145            1 

In [100]:
#from target and tcga data
IPD_metadata$vital_status[IPD_metadata$vital_status == ""] <- NA
IPD_metadata$vital_status[IPD_metadata$vital_status == "alive"] <- 1
IPD_metadata$vital_status[IPD_metadata$vital_status == "dead"] <- 0
IPD_metadata$vital_status[IPD_metadata$vital_status == "not reported"] <- NA

define age classes

In [101]:
IPD_metadata$age_class[is.na(IPD_metadata$age_at_diagnosis)] <- "Unknown"
IPD_metadata$age_class[which(IPD_metadata$age_at_diagnosis < 2)] <- "Infants"
IPD_metadata$age_class[which(IPD_metadata$age_at_diagnosis > 2 & IPD_metadata$age_at_diagnosis < 12)] <- "Children"
IPD_metadata$age_class[which(IPD_metadata$age_at_diagnosis > 12 & IPD_metadata$age_at_diagnosis < 16)] <- "Adolescents"
IPD_metadata$age_class[which(IPD_metadata$age_at_diagnosis > 16 & IPD_metadata$age_at_diagnosis < 25)] <- "Young adults"
IPD_metadata$age_class[which(IPD_metadata$age_at_diagnosis > 25)] <- "Adults"

Clean up some minor typos

In [102]:
IPD_metadata$disease_type[IPD_metadata$disease_type == "ETMR_"] <- "ETMR"
IPD_metadata$disease_type[IPD_metadata$disease_type == "PA_"] <- "PA"
IPD_metadata$disease_type[IPD_metadata$disease_type == "HGG_K27M_K27M"] <- "HGG_K27M"
IPD_metadata$disease_type[IPD_metadata$disease_type == "MB_WNT_WNT"] <- "MB_WNT"
IPD_metadata$disease_type[IPD_metadata$disease_type == "MB_SHH_SHH"] <- "MB_SHH"
IPD_metadata$disease_type[IPD_metadata$disease_type == "MB_Group3_Group3"] <- "MB_Group3"
IPD_metadata$disease_type[IPD_metadata$disease_type == "MB_Group4_Group4"] <- "MB_Group4"

In [103]:
table(IPD_metadata$group)


 CBTTC   DKFZ TARGET   TCGA 
   969    199    157   3204 

In [104]:
table(IPD_metadata$recurrence)


  0   1 
758 401 

In [105]:
table(IPD_metadata$group)


 CBTTC   DKFZ TARGET   TCGA 
   969    199    157   3204 

In [106]:
table(IPD_metadata$sample_type)


                       Additional Metastatic 
                                           1 
                           Initial CNS Tumor 
                                         730 
                                  Metastatic 
                                         369 
                                     Primary 
                                         196 
                               Primary Tumor 
                                        2786 
                                 Progressive 
                                         153 
                                  Recurrence 
                                          70 
Recurrent Blood Derived Cancer - Bone Marrow 
                                           1 
                             Recurrent Tumor 
                                          46 
                                     Relapse 
                                           3 
                           Second Malignancy 
                                 

In [107]:
save(IPD_metadata, file = paste0(manifestpath,"Master_manifest.RData"))